In [1]:
from curobo.wrap.reacher.ik_solver import IKSolver, IKSolverConfig
from curobo.types.robot import RobotConfig

import torch
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True

kinematics_fused_cu not found, JIT compiling...
geom_cu binary not found, jit compiling...
lbfgs_step_cu not found, JIT compiling...
line_search_cu not found, JIT compiling...
tensor_step_cu not found, jit compiling...


In [2]:
import os

ik_solver_config = IKSolverConfig.load_from_robot_config(
    robot_cfg=RobotConfig.from_basic(
        f"{os.getcwd()}/robots/piper_ros/src/piper_description/urdf/piper_description.urdf",
        base_link="base_link",
        ee_link="link8",
    ),
    position_threshold=0.005,
    self_collision_check=False,
    self_collision_opt=False,
    regularization=False,
    use_cuda_graph=True,
)

ik_solver = IKSolver(ik_solver_config)

In [3]:
import torch

q = torch.full((1, ik_solver.dof), fill_value=0., device="cuda")
fk_result = ik_solver.fk(q)
fk_result.ee_pose.get_matrix()

tensor([[[-8.7093e-02,  9.9620e-01,  7.4804e-06,  1.9143e-01],
         [ 1.1593e-05, -6.5732e-06,  1.0000e+00, -1.4386e-06],
         [ 9.9620e-01,  8.7093e-02, -1.1048e-05,  2.2502e-01],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00]]],
       device='cuda:0')

In [4]:
%%timeit -n 10
fk_result = ik_solver.fk(q)
fk_result.ee_pose.get_matrix()

The slowest run took 26.38 times longer than the fastest. This could mean that an intermediate result is being cached.
784 μs ± 1.49 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [5]:
%%timeit -n 10

ik_result = ik_solver.solve_batch(fk_result.ee_pose)
# ik_result.solution

The slowest run took 15.26 times longer than the fastest. This could mean that an intermediate result is being cached.
13 ms ± 20.4 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
